# <b>Required Imports</b>

In [1]:
import pickle,sys,os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.7.0


In [2]:
!wget https://github.com/GastonMazzei/TraceMeOut/blob/main/processed_trace/Dataset0.pkl?raw=true
!wget https://github.com/GastonMazzei/TraceMeOut/blob/main/processed_trace/Dataset1.pkl?raw=true
!wget https://github.com/GastonMazzei/TraceMeOut/blob/main/processed_trace/Dataset2.pkl?raw=true
!wget https://github.com/GastonMazzei/TraceMeOut/blob/main/processed_trace/Dataset3.pkl?raw=true

--2021-12-30 17:30:39--  https://github.com/GastonMazzei/TraceMeOut/blob/main/processed_trace/Dataset0.pkl?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/GastonMazzei/TraceMeOut/raw/main/processed_trace/Dataset0.pkl [following]
--2021-12-30 17:30:39--  https://github.com/GastonMazzei/TraceMeOut/raw/main/processed_trace/Dataset0.pkl
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/GastonMazzei/TraceMeOut/main/processed_trace/Dataset0.pkl [following]
--2021-12-30 17:30:40--  https://raw.githubusercontent.com/GastonMazzei/TraceMeOut/main/processed_trace/Dataset0.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubuser

In [3]:
!ls

'Dataset0.pkl?raw=true'  'Dataset2.pkl?raw=true'   sample_data
'Dataset1.pkl?raw=true'  'Dataset3.pkl?raw=true'


In [4]:
# with open('Dataset0.pkl?raw=true','rb') as f:
#   data = pickle.load(f)

# NEW! We load data from all processors! :-)
datas = {}
for n in [0,1,2,3]:
  with open(f'Dataset{n}.pkl?raw=true','rb') as f:
    datas[n] = pickle.load(f)

#<b>Configuration File: architecture & hyperparameters</b>

(<i>If the datasets change in the repo, non-architectural information should be updated using the configuration.py file. Most probably an error will be raised because of size mismatch if this is not updated.</i>)

In [5]:

# Model-specific parameters
T=8 # duration of the window in dt units
dt = 100000 # time in microseconds
UNIQUES=3807  #number of unique ids
MI=5709  #max number of interactions
ML=5712  #max number of leaves
NCATEGORIES=2

# Architectural parameters
ACT1 = 'relu'
FILTERS1 = 4
KSIZE1 = (2,1)
PSIZE1 = (max([T//4,2]),)
NDENSE1 = 8
DROP1 = 0.4

ACT2 = 'relu'
FILTERS2 = 4
KSIZE2 = (2,2)
PSIZE2 = (max([T//2,2]),1)
stride = (1,1)
NDENSE2 = 8
DROP2 = 0.75

ACT3='relu'
NDENSE3=8
DROP3 = 0.4

ACT4='relu'
NDENSE4=8
DROP4 = 0.2

# Training parameters
VAL=0.25
BATCH=10
EPOCHS=50
L=5 # a length used to generate random data just for testing
LR=0.05

# Extras
POOLING = False
PROCS=[3, 2, 0, 1]

In [6]:
# Assert the dataset's keys and the procs in the configuration are the same :-)
temp1, temp2 = sorted(PROCS), sorted(list(datas.keys()))
assert(temp1 == temp2)
NPROCS = len(PROCS)

# define two sets of inputs: 
input_shape_flavours = (BATCH, T, ML, 1)
input_shape_structure = (BATCH, T, MI, 2)

def produce_model():
	inputFlavours = tf.keras.Input(shape=input_shape_flavours[1:])
	inputStructure = tf.keras.Input(shape=input_shape_structure[1:])

	# the first branch operates on the first input (https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D)
	x = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS1,         KSIZE1,       (1,1),      'valid',  activation = ACT1,
				input_shape = input_shape_flavours[1:]
				)(inputFlavours)
	x = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS1 * 2,         KSIZE1,       (1,1),      'valid',  activation = ACT1,
				)(x)
	x = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS1 * 2,         KSIZE1,       (1,1),      'valid',  activation = ACT1,
				)(x)
	#x = tf.keras.layers.MaxPool1D(pool_size=PSIZE1)(x)
	x = tf.keras.layers.Flatten()(x)
	x = tf.keras.layers.Dropout(DROP1)(x)
	x = tf.keras.layers.Dense(NDENSE1, activation = ACT1)(x)
	x = tf.keras.layers.Dropout(DROP1)(x)
	x = tf.keras.layers.Dense(NDENSE1 // 2, activation = ACT1)(x)
	x = tf.keras.layers.Dropout(DROP1)(x)
	x = tf.keras.layers.Dense(NDENSE1 // 2 // 2, activation = ACT1)(x)
	x = tf.keras.Model(inputs = inputFlavours, outputs=x)

	# the second branch opreates on the second input (https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
	y = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS2,         KSIZE2,    stride,      'valid',  activation = ACT2,
				input_shape = input_shape_structure[1:]
				)(inputStructure)
	y = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS2 * 2,         KSIZE2,       stride,      'valid',  activation = ACT2,
				)(y)
	#y = tf.keras.layers.MaxPool2D(pool_size=PSIZE2)(y)
	y = tf.keras.layers.Conv2D(
				# Filters, Kersize, Strides, Padding,  Activation
				FILTERS2 * 2,         KSIZE2,       stride,      'valid',  activation = ACT2,
				)(y)
	y = tf.keras.layers.MaxPool2D(pool_size=PSIZE2)(y)
	y = tf.keras.layers.Flatten()(y)
	y = tf.keras.layers.Dropout(DROP2)(y)
	y = tf.keras.layers.Dense(NDENSE2, activation = ACT2)(y)
	y = tf.keras.layers.Dropout(DROP2)(y)
	y = tf.keras.layers.Dense(NDENSE2 // 2, activation = ACT2)(y)
	y = tf.keras.layers.Dropout(DROP2)(y)
	y = tf.keras.layers.Dense(NDENSE2 // 2 // 2, activation = ACT2)(y)
	y = tf.keras.Model(inputs = inputStructure, outputs=y)



	# combine the output of the two branches
	combined = tf.keras.layers.concatenate([x.output, y.output])
	z = tf.keras.layers.Dense(NDENSE3, activation = ACT3)(combined)
	z = tf.keras.layers.Dropout(DROP3)(z)
	z = tf.keras.layers.Dense(NDENSE3, activation = ACT3)(z)
	z = tf.keras.layers.Dropout(DROP3)(z)
	z = tf.keras.layers.Dense(NDENSE3, activation = ACT3)(z)

	# our model will accept the inputs of the two branches and
	# then output a single value
	return tf.keras.Model(inputs=[x.input, y.input], outputs=z)

models = []
for n in range(NPROCS):
	models += [produce_model()]
TOTALINPUTS = []
for m in models:
	TOTALINPUTS += [m.inputs[0], m.inputs[1]]
combined = tf.keras.layers.concatenate([m.output for m in models])
w = tf.keras.layers.Dense(NDENSE4, activation = ACT4)(combined)
w = tf.keras.layers.Dropout(DROP4)(w)
w = tf.keras.layers.Dense(NDENSE4, activation = ACT4)(combined)
w = tf.keras.layers.Dropout(DROP4)(w)
w = tf.keras.layers.Dense(NDENSE4, activation = ACT4)(combined)
w = tf.keras.layers.Dense(NCATEGORIES, activation="softmax")(w)
model = tf.keras.Model(inputs=TOTALINPUTS, outputs=w)

# Compile the model :-)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=LR),
							loss=tf.keras.losses.CategoricalCrossentropy(),
							metrics=[
											#tf.keras.metrics.CategoricalCrossentropy(),
											tf.keras.metrics.CategoricalAccuracy(),
											#tf.keras.metrics.AUC(),
											])

# Print input size
print(model.summary())
IShape = model.input_shape
OShape = model.output_shape

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 8, 5709, 2)  0           []                               
                                ]                                                                 
                                                                                                  
 input_4 (InputLayer)           [(None, 8, 5709, 2)  0           []                               
                                ]                                                                 
                                                                                                  
 input_6 (InputLayer)           [(None, 8, 5709, 2)  0           []                               
                                ]                                                          

#<b>Building the dataset generators to feed the Neural Net</b>

In [7]:
def trimmer_and_recomputer(datas):
  global BATCH
  l = []
  for n in datas.keys():
    l += [len(datas[n]['X1'])]
  prune_length = min([len(d['X1']) for d in datas.values()])
  for n in datas.keys():
    new = {}
    for k in datas[n].keys():
      new[k] = datas[n][k][:prune_length]
    datas[n] = new.copy()
  L = prune_length
  LTR = int(L*(1-VAL))
  print(f"Old batch size was: {BATCH}")
  B = 32
  LTR = LTR//B * B # LTR is approximated to the closest multiple of B :-) which is convenient for GPU's given their architecture
  LVA = L - LTR
  LVA = LVA // B * B # THe same for LVA
  BATCH = (BATCH // B + 1) * B # And the same for Batch Size
  print(f"New batch size is: {BATCH}, LVA and LTR are: {LVA}, {LTR} and L is {L}, ... does L>=LTR+LVA? {L>=(LTR+LVA)}")
  return BATCH, L, LVA, LTR


def build_data(datas,n):
  # Open data
  data = datas[n]
  X1, X20, Y = data['X1'],data['X2'],data['Y']

  # Process Y
  ONE_HOT_Y = np.zeros((len(Y),2))
  for i in range(len(Y)):
      ONE_HOT_Y[i,Y[i]] = 1
  ONE_HOT_Y = ONE_HOT_Y.astype('float32')

  # Process X1
  X1_TRACKER = []
  for i_,X_ in enumerate(X1):
      X1_TRACKER.append((ML-len(X_)))
      X1[i_] = [float(x) / UNIQUES for x in X_]

  # Process X2
  X2 = [[] for _ in range(len(X20))]
  X2_TRACKER = []
  for i,X_ in enumerate(X20):
      c = 0
      for i_,y_ in enumerate(X_):
          for z in y_:
              X2[i] += [[float(i_)/ML,float(z)/ML]]
              c += 1
      X2_TRACKER.append(MI-c) 

  # Record the results
  datas[n]['X1'] = (X1, X1_TRACKER)
  datas[n]['X2'] = (X2, X2_TRACKER)
  datas[n]['Y'] = (ONE_HOT_Y,)

  return


def produce_data(A,B):
    w = list(range(A,B))
    np.random.shuffle(w)
    YS = datas[PROCS[0]]['Y'][0]
    for i in w:
            yield (
                    tuple([
                  ( tf.convert_to_tensor(np.asarray([ (datas[nproc]['X1'][0][j] + [0.] * datas[nproc]['X1'][1][j]) for j in range(i-T+1,i+1)]).reshape(1,T,-1,1)), 
                tf.convert_to_tensor(np.asarray([ (datas[nproc]['X2'][0][j] + [[0., 0.]] * datas[nproc]['X2'][1][j]) for j in range(i-T+1,i+1)]).reshape(1,T,-1,2)),
              ) for nproc in range(len(PROCS))
                        ]), 
                    tf.convert_to_tensor(YS[i:i+1,:].reshape(1,2)),
                  )
            

BATCH, L, LVA, LTR = trimmer_and_recomputer(datas)
for n in PROCS:
  build_data(datas,n)

OS = (
        tuple([
          (tf.TensorSpec(shape=(None,T,ML,1), dtype=tf.float32),
          tf.TensorSpec(shape=(None,T,MI,2), dtype=tf.float32)
          ) for nproc in range(len(PROCS)) 
          ]),
          tf.TensorSpec(shape=(None,NCATEGORIES), dtype=tf.float32),
    )

print(f'Finished building the output structure')
trainD = tf.data.Dataset.from_generator(lambda: produce_data(T,LTR), output_signature=OS)#output_types=(tf.float32), output_shapes=OS)
print(f'Finished building the training dataset')
valD = tf.data.Dataset.from_generator(lambda: produce_data(LTR+T,L), output_signature=OS)# output_types=(tf.float32), output_shapes=OS)
print(f'Finished building the validation dataset')


Old batch size was: 10
New batch size is: 32, LVA and LTR are: 224, 608 and L is 852, ... does L>=LTR+LVA? True
Finished building the output structure
Finished building the training dataset
Finished building the validation dataset


#<b>Training the Neural Net</b>

In [8]:
print(f'About to train! :-)')
history = model.fit(trainD, epochs=EPOCHS, batch_size=BATCH, validation_data=valD, verbose=2)
print('Finished training! :-)')

About to train! :-)
Epoch 1/50
600/600 - 199s - loss: 0.6367 - categorical_accuracy: 0.7217 - val_loss: 0.7627 - val_categorical_accuracy: 0.5508 - 199s/epoch - 332ms/step
Epoch 2/50
600/600 - 181s - loss: 0.5966 - categorical_accuracy: 0.7233 - val_loss: 0.7638 - val_categorical_accuracy: 0.5508 - 181s/epoch - 302ms/step
Epoch 3/50
600/600 - 181s - loss: 0.5969 - categorical_accuracy: 0.7233 - val_loss: 0.7503 - val_categorical_accuracy: 0.5508 - 181s/epoch - 302ms/step
Epoch 4/50
600/600 - 181s - loss: 0.5946 - categorical_accuracy: 0.7233 - val_loss: 0.7144 - val_categorical_accuracy: 0.5508 - 181s/epoch - 301ms/step
Epoch 5/50
600/600 - 178s - loss: 0.5939 - categorical_accuracy: 0.7233 - val_loss: 0.7623 - val_categorical_accuracy: 0.5508 - 178s/epoch - 297ms/step
Epoch 6/50
600/600 - 179s - loss: 0.5942 - categorical_accuracy: 0.7233 - val_loss: 0.7626 - val_categorical_accuracy: 0.5508 - 179s/epoch - 298ms/step
Epoch 7/50
600/600 - 178s - loss: 0.5958 - categorical_accuracy: 0.7

KeyboardInterrupt: ignored

#<b>Plotting the Neural Net results</b>

In [ ]:
# Display results
f,ax = plt.subplots(1,2,figsize=(15,10))

ax[0].plot(history.history['loss'],label='loss', c='k', lw=2)
ax[0].plot(history.history['val_loss'], label='val loss', c='r', lw=2)
ax[0].grid()
ax[0].set_title('Training and Validation Loss* over the Epochs\n*TensorFlow categorical crossentropy')
ax[0].set_ylim(0,None)
ax[0].legend()

ax[1].plot(history.history['categorical_accuracy'],label='accuracy', c='k', lw=2)
ax[1].plot(history.history['val_categorical_accuracy'], label='val accuracy', c='r', lw=2)
ax[1].set_ylim(0,1)
ax[1].hlines(0.5, 0, EPOCHS, color='y', ls=':', lw=4, label='Null Hypothesis\n(perfect 50% tag balance case)')
ax[1].grid()
ax[1].set_title('Training and Validation Acurracy over the Epochs')
ax[1].legend()																																																																																																																																																																				

plt.show()

In [ ]:
# Print the raw values of the metrics, just in case we want to use them without re-running the notebook :O
print(history.history['loss'])
print(history.history['val_loss'])
print(history.history['categorical_accuracy'])
print(history.history['val_categorical_accuracy'])